In [6]:
#dependencies and set up
import pandas as pd 
from bs4 import BeautifulSoup as bs
import requests 
from splinter import Browser
import time
from urllib.parse import urljoin
#set up Chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [2]:
#visit the nasa website with chromedriver
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

#create the soup
html = browser.html
nasa_soup = bs(html, 'html.parser')

In [3]:
time.sleep(1)

In [4]:
latest_article = nasa_soup.find('li', class_='slide')
latest_title = latest_article.find('div', class_='content_title').text
news_p = latest_article.a.text
print(latest_title)
print(news_p)

NASA's Perseverance Pays Off Back Home
Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.NASA's Perseverance Pays Off Back Home


In [5]:
# articles = nasa_soup.find('li', class_='slide')
# for article in articles: 
#     title = article.find('div', class_='content_title').text
#     p = article.a.text
# #     latest_paragraph = article.
#     print(f'Title: {title} \n')
#     print(f'Paragraph: {p}')
#     print('_______________')

In [16]:
browser.quit()

In [7]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [7]:
#Visit JPL website
# jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# NOTE: JPL's website redesigned where images are located... the url below is the new location to find latest mars featured images
jpl_url = 'https://mars.nasa.gov/multimedia/images/?page=0&per_page=25&order=pub_date+desc&search=&condition_1=1%3Ais_in_resource_list&category=51'
browser.visit(jpl_url)
#create soup for jpl site 
time.sleep(2)
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

#navigate to the featured image's page 
item = jpl_soup.find('ul', class_='item_list')
latest_item = item.li
pic_href = latest_item.a['href']
browser.click_link_by_href(pic_href)
#create soup for the new page and store image url in new variable
jpl_image_page = browser.html
jpl_image_page_soup = bs(jpl_image_page, 'html.parser') 
image_anchor = jpl_image_page_soup.find('a', class_='fancybox')
image_link = image_anchor.find('img')['src']

#use urljoin to get full image url
base = 'https://mars.nasa.gov'
featured_image_url = urljoin(base, image_link)
featured_image_url

C:\Users\im022\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:474: FutureWarning: browser.find_link_by_href is deprecated. Use browser.links.find_by_href instead.
  warnings.warn(


'https://mars.nasa.gov/system/resources/detail_files/25564_spinoff-1200.jpg'

In [9]:
#load page for mars facts 
mars_facts = 'https://space-facts.com/mars/'
browser.visit(mars_facts)
facts_html = browser.html

In [10]:
# scrape mars facts using pandas
facts = pd.read_html(facts_html)
table = facts[0]
#write table to html 
table_html = table.to_html()


In [11]:
#visit usgs site 
usgs = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs)
usgs_html = browser.html 
usgs_soup = bs(usgs_html)
    


In [12]:
#make list of links to visit 
links = usgs_soup.find_all('div', class_='item')
links_list = []
for link in links:
    links_list.append(link.h3.text)
links_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [13]:
hemisphere_urls = []
for link in links_list:
    browser.click_link_by_partial_text(link)
    browser.click_link_by_partial_text('Sample')
    image_link = browser.url
    info_dict = dict({"title":link, "img_url":image_link})
#     print(info_dict)
    hemisphere_urls.append(info_dict)
    browser.visit(usgs)

C:\Users\im022\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


In [14]:
#check to see if all info was appended to hemisphere_urls 
hemisphere_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'}]

In [15]:
# Mongo